In [1]:
from problem_model import forest_problem_vaaler, simple_forest_problem
from desdeo.problem import simple_knapsack, Problem
from methods import binary_nsga3, binary_rvea
from desdeo.emo.hooks.archivers import FeasibleArchive
from desdeo.emo.operators.selection import ReferenceVectorOptions
from desdeo.mcdm import rpm_solve_solutions
from desdeo.tools import payoff_table_method, PyomoBonminSolver, PyomoCBCSolver
from desdeo.tools.non_dominated_sorting import fast_non_dominated_sort
import polars as pl
import numpy as np
import plotly.graph_objects as go
from pathlib import Path

In [4]:
problem = Problem.load_json(Path("../data/vaaler.json"))



In [5]:

# solver, publisher = binary_rvea(problem=problem, n_generations=500, reference_vector_options=ReferenceVectorOptions(number_of_vectors=1000), forced_verbosity=2)
solver, publisher = binary_rvea(problem=problem)

archive = FeasibleArchive(problem=problem, publisher=publisher)
publisher.auto_subscribe(archive)

In [6]:
emo_result = solver()

In [1]:
#fronts = fast_non_dominated_sort(fitness)
#fronts = [np.where(fronts[i])[0] for i in range(len(fronts))]
# non_dominated = fronts[0]

temp = archive.feasible_archive[["NPV", "DWV"]].to_numpy()
fronts = fast_non_dominated_sort(temp)
fronts = [np.where(fronts[i])[0] for i in range(len(fronts))]
non_dominated = fronts[0]
non_dom_indices = non_dominated

non_dom_solutions = archive.feasible_archive[non_dom_indices]

NameError: name 'archive' is not defined

In [28]:
ref_point = {"NPV": 2165, "DWV": 5690}

ideal = {"NPV": archive.feasible_archive["NPV"].max(), "DWV": archive.feasible_archive["DWV"].max()}
nadir = {"NPV": archive.feasible_archive["NPV"].min(), "DWV": archive.feasible_archive["DWV"].min()}

print(f"{ideal=}")
print(f"{nadir=}")
problem = problem.update_ideal_and_nadir(new_ideal=ideal, new_nadir=nadir)

problem.get_constraint("row_constraint_00")

mcdm_result = rpm_solve_solutions(problem, reference_point=ref_point, solver=PyomoCBCSolver)
mcdm_result

ideal={'NPV': 6584.0, 'DWV': 6180.0}
nadir={'NPV': 15.0, 'DWV': 4980.0}


[SolverResults(optimal_variables={'X_1': [[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0]], 'X_2': [[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [0.0, 1.0, 0.0, 0.0]], 'X_3': [[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [0.0, 1.0, 0.0, 0.0]], 'X_4': [[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0]], 'X_5': [[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0]], '_alpha': -0.275}, optimal_objectives={'NPV': 4109.0, 'DWV': 6020.0}, constraint_values={'row_constraint_00': 0.0, 'row_constraint_00_': 0.0, 'row_constraint_01': 0.0, 'row_constraint_01_': 0.0, 'row_constraint_02': 0.0, 'row_constraint_02_': 0.0, 'row_constraint_10': 0.0, 'row_constraint_10_': 0.0, 'row_constraint_11': 0.0, 'row_constraint_11_': 0.0, 'row_constraint_12': 0.0, 'row_constraint_12_': 0.0, 'row_constraint_20': 0.0, 'row_constraint_20_': 0.0, 'row_constraint_21': 0.0, 'row_constraint_21_': 0.0, 'row_constraint_22': 0.0, 'row_constraint_22_': 0.0, 'row_constraint_30': 0.0,

In [29]:
fig = go.Figure(
    go.Scatter(
        x=emo_result.outputs["NPV"],
        y=emo_result.outputs["DWV"],
        mode="markers",
        marker=dict(size=6, color="blue"),
        name="raw"
    )
)
fig.add_trace(
    go.Scatter(
        x=non_dom_solutions["NPV"],
        y=non_dom_solutions["DWV"],
        mode="markers",
        marker=dict(size=10, color="red", opacity=0.5),
        name="feasible non-dominated"
    )
)
fig.add_trace(
    go.Scatter(
        x=[r.optimal_objectives["NPV"] for r in mcdm_result],
        y=[r.optimal_objectives["DWV"] for r in mcdm_result],
        mode="markers",
        marker=dict(size=8, color="green"),
        name="PO"
    )
)

In [12]:
1 in solutions

True

In [13]:
0 in solutions

True

In [69]:
result.solutions

X_1,X_2,X_3,X_4
f64,f64,f64,f64
1.0,1.0,1.0,1.0
1.0,1.0,1.0,1.0
1.0,1.0,1.0,1.0
1.0,1.0,1.0,1.0
1.0,1.0,1.0,1.0
…,…,…,…
1.0,1.0,1.0,1.0
1.0,1.0,1.0,1.0
1.0,1.0,1.0,1.0
